# 目標：

1. 理解Dense Layer的輸入/輸出資料大小。
2. 能簡單的利用Dense Layer來建立模型。
3. 理解Convolution/Pooling Layer的輸入/輸出資料大小。

### [A. 測試Dense Layer I/O, 並以Dense Layer建立模型](#A.-%E6%B8%AC%E8%A9%A6Dense-Layer-I/O,-%E4%B8%A6%E4%BB%A5Dense-Layer%E5%BB%BA%E7%AB%8B%E6%A8%A1%E5%9E%8B%E3%80%82)
* [Sequential model: Logistic Regression](#Sequential-model:-Logistic-Regression)
* [Sequential model: Softmax Regression](#Sequential-model:-Softmax-Regression)
* [練習：建立Multilayer Perceptron的模型，並且丟亂數資料進去做訓練](#%E7%B7%B4%E7%BF%92%EF%BC%9A%E5%BB%BA%E7%AB%8BMultilayer-Perceptron%E7%9A%84%E6%A8%A1%E5%9E%8B%EF%BC%8C%E4%B8%A6%E4%B8%94%E4%B8%9F%E4%BA%82%E6%95%B8%E8%B3%87%E6%96%99%E9%80%B2%E5%8E%BB%E5%81%9A%E8%A8%93%E7%B7%B4%E3%80%82)

### [B. 測試Convolutional Layer I/O](#B.-%E6%B8%AC%E8%A9%A6Convolutional-Layer-I/O%E3%80%82)
### [C. 測試Max Pooling Layer I/O](#C.-%E6%B8%AC%E8%A9%A6Max-Pooling-Layer-I/O%E3%80%82)

---

In [ ]:
# # =====================================================================
# # 由於課堂上可能有多人共用同一顆GPU，以下限定使用者只能用計算卡上面一半的記憶體。
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.5 # 使用一半記憶體
# set_session(tf.Session(config=config))
# # =====================================================================

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D
from keras.optimizers import SGD

---

## A. 測試Dense Layer I/O, 並以Dense Layer建立模型。

### Sequential model: Logistic Regression

In [ ]:
import pandas as pd

In [ ]:
# pd.Series(np.random.normal(0,1,(500,300))
#           .reshape(-1)).plot(kind="hist",bins=100)

In [ ]:
randData=np.random.normal(0,1,(5,3))  # normal分佈的亂數資料當input, 
                                      # 5個樣本，每個樣本有3個特徵

model = Sequential()                  # 定義模型
model.add(Dense(1, input_shape=(3,),activation='sigmoid') ) 
# 模型加入只有一個神經元的Dense層，且用sigmoid啟用，輸出機率

model.predict(randData)  # 應該會回傳5個介於0和1之間的數值。分別為各樣本的預測機率

[回到頂部](#%E7%9B%AE%E6%A8%99%EF%BC%9A)

---

### Sequential model: Softmax Regression

In [ ]:
randData=np.random.normal(0,1,(5,3))         # normal分佈的亂數資料當input, 5個樣本，每個樣本有3個特徵

model = Sequential()
model.add(Dense(3,input_shape=(3,), activation='softmax') )   # 定義3個神經元，並以Softmax啟用，輸出機率
print( model.predict(randData) )             # 往前傳遞
print()
print( model.predict(randData).sum(axis=1) ) # 驗證softmax輸出：P_A+P_B+P_C=1

[回到頂部](#%E7%9B%AE%E6%A8%99%EF%BC%9A)

---

### 練習：建立Multilayer Perceptron的模型，並且將$X_{new}$, $y_{new}$丟進去做訓練。

In [ ]:
import numpy as np

In [ ]:
X=np.zeros((90,3))
for j in range(0,30):
    X[j,:]=0.
for j in range(30,60):
    X[j,:]=1.
for j in range(60,90):
    X[j,:]=2.
y=X[:,0]

In [ ]:
X[:,0].mean()

In [ ]:
X[:,0].std()

In [ ]:
# data standarization
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler=scaler.fit(X)
X_new=scaler.transform(X)

http://sebastianraschka.com/Articles/2014_about_feature_scaling.html

In [ ]:
X_new[:,0].mean()

In [ ]:
X_new[:,0].std()

In [ ]:
# convert y into one-hot representation
from keras.utils import to_categorical
y_new=to_categorical(y)

print("X_new=\n",X_new)
print()
print("y_new=\n",y_new)

In [ ]:
#練習於此
model=Sequential()
model.add( Dense(10000,activation="relu",input_shape=(3,)) )
model.add( Dense(32,activation="relu") )
model.add( Dense(3,activation="softmax"))

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import SGD

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=SGD(lr=0.01),
              metrics=["accuracy"])

In [ ]:
history=model.fit(X_new,y_new,epochs=50)

In [ ]:
pd.Series(history.history["loss"]).plot()

[回到頂部](#%E7%9B%AE%E6%A8%99%EF%BC%9A)

---

## B. 測試Convolutional Layer I/O。

### ```padding='valid'```

In [ ]:
randData=np.random.normal(0,1,(10,5,5,3))  # normal分佈的亂數資料當input, 10個3D樣本

model = Sequential()
model.add(Conv2D(filters=96, kernel_size=(3, 3),
                 strides=(1,1),
                 padding='valid',
                 input_shape=(5,5,3)
                )
         )
print( model.predict(randData).shape )    # 看輸出資料的形狀

In [ ]:
5-3+1

### ```padding='same'```

In [ ]:
randData=np.random.normal(0,1,(10,12,12,3))  # normal分佈的亂數資料當input, 10個3D樣本

model = Sequential()
model.add(Conv2D(filters=96, kernel_size=(3, 3),
                 strides=(3,3),
                 padding='same',
                 input_shape=(None,None,3)
                )
         )
print( model.predict(randData).shape )    # 看輸出資料的形狀

* 若strides=(1,1)且```padding='same'```，則表示程式會利用padding來確保input data size和output data size相等。
* 若你想知道```padding='same'```和```padding='valid'```的公式，可見：https://www.tensorflow.org/api_guides/python/nn#Convolution

```padding=valid```: $w' = \frac{w+2p-f}{s}+1$

```padding=same```: $w' = w/s$

[回到頂部](#%E7%9B%AE%E6%A8%99%EF%BC%9A)

## C. 測試Max Pooling Layer I/O。

In [ ]:
randData=np.random.normal(0,1,(10,4,4,3))  # normal分佈的亂數資料當input, 10個3D樣本
model = Sequential()
model.add(MaxPooling2D(pool_size=(2, 2),
                       strides=(2,2),
                       input_shape=(4,4,3)
                      )
         )
print( model.predict(randData).shape )    # 看輸出資料的形狀

[回到頂部](#%E7%9B%AE%E6%A8%99%EF%BC%9A)